<a href="https://colab.research.google.com/github/yukinaga/bayesian_statistics/blob/main/section_5/01_bayes_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  ベイズ推定による線形回帰
線形回帰は、教師あり学習の一種で、変数間の関係を予測します。  
今回は、ベイズ推定による線形回帰をPyMC3により実装します。

## pymc3のインストール
pymc3をpipによりインストールします。

In [ ]:
!pip install pymc3

必要な各設定を行います。

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import datasets
import pymc3 as pm

plt.style.use("seaborn-darkgrid")
plt.rcParams["figure.figsize"] = [12, 9]  # グラフの大きさ

## データセットの読み込み
3品種、合計150個の花のデータ「Iris dataset」を読み込みます。  

In [ ]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df.head()

各列のラベルの意味は、`DESCR`により表示できます。

In [ ]:
print(iris.DESCR)  # データセットの説明

seabornを使い、データの関係性を可視化します。

In [ ]:
sns.pairplot(iris_df)

データセットの特徴を把握するために、`describe()`で統計量を表示します。



In [ ]:
iris_df.describe()

## ●単回帰
単回帰では、直線を使い1つの説明変数で目的変数を予測します。  
$x$を説明変数、$y$を目的変数、$a$を係数、$b$を切片としたとき、単回帰は以下の式で表されます。  
$$y = ax + b$$

以下のコードでは、linear_model.LinearRegressionにより線形回帰のモデルを生成し、fitメソッドにより、モデルの訓練を行います。  
訓練の結果、式の係数と切片が最適化されます。  


In [ ]:
from sklearn import linear_model

x = iris_df["sepal length (cm)"]  # 額の長さ
y = iris_df["petal length (cm)"]  # 花弁の長さ

lm = linear_model.LinearRegression() # 線形回帰モデル
lm.fit(x.values.reshape(-1, 1), y.values.reshape(-1, 1))  # モデルの訓練

訓練済みのモデルから、式の係数と切片を取得します。

In [ ]:
a = lm.coef_[0][0] # 係数
b = lm.intercept_[0] # 切片
print("a: ", a) 
print("b: ", b) 

取得した係数と切片を使った回帰直線を、元のデータとともにグラフで表示します。

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x, y)

y_reg = a * x + b  # 回帰直線
plt.plot(x, y_reg, c="red") 

plt.xlabel("sepal length (cm)")
plt.ylabel("petal length (cm)")
plt.show()

## PyMC3による線形回帰



PyMC3により、線形回帰の統計モデルを実装します。  
この場合、事前分布には係数、切片、データのばらつき具合の3つがあります。  




In [ ]:
with pm.Model() as model:
    
    # 事前分布
    a_n = pm.Normal("a", a, 10)  # 係数
    b_n = pm.Normal("b", b, 10)  # 切片
    e = pm.Exponential("error", 1)  # データのばらつき
    
    obs = pm.Normal("likehood", a_n*x + b_n, e, observed=y)
    trace = pm.sample(4000, chains=2, return_inferencedata=False)

pm.summary()によりモデルの概要を表示します。

In [ ]:
with model:
    print(pm.summary(trace, hdi_prob=0.95))

plot_posteriorにより、PyMC3によって推論された事後分布をグラフで表示します。

In [ ]:
with model:
    pm.plot_posterior(trace, point_estimate='mode')

`pm.plot_trace()`によりサンプリング結果を可視化します。

In [ ]:
with model:
    pm.plot_trace(trace)

事後分布から、`sample_posterior_predictive()`によりサンプリングを行います。

In [ ]:
with model:
    y_p = pm.sample_posterior_predictive(trace, samples=5000, model=model)

y_pred = y_p["likehood"]

データの形状を確認します。

In [ ]:
y_pred.shape

グラフを信頼区間と共に描画します。

In [ ]:
# 係数と切片の推定
a_m = trace["a"].mean()
b_m = trace["b"].mean()

# 信頼区間の設定
x_i = np.argsort(x)
h_95 = pm.hdi(y_pred, hdi_prob=0.95)[x_i]
x_h = x[x_i]

plt.scatter(x, y, label="data")
plt.plot(x_h, a_m*x_h+b_m, c = "red", label="y=ax+b")
plt.fill_between(x_h, h_95[:,0], h_95[:,1], alpha=0.25, label="95hpd")

plt.legend()
plt.show()